In [282]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine 
import os
import json
import re
from ast import literal_eval

In [283]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [284]:
def read_data():
    query = "SELECT distinct i.tableId, i.rowIndex, ip.rowCounter, i.subvec_std, ip.subvec_std_issue_parsed FROM issues AS i LEFT JOIN issues_parsed AS ip ON i.tableId = ip.tableId AND i.rowIndex = ip.rowIndex;"
    with engine.connect() as conn:
        df = pd.read_sql(query, conn)
    for idx, row in enumerate(df.itertuples()):  ###https://stackoverflow.com/questions/43222878/iterate-over-pandas-dataframe-and-update-the-value-attributeerror-cant-set-a
        if pd.isna(row.subvec_std):
            df.loc[idx, 'subvec'] = row.subvec_std_issue_parsed
        else:
            df.loc[idx, 'subvec'] = row.subvec_std
    df.subvec = df.subvec.apply(lambda s: literal_eval(str(s)))
    return df.where(pd.notnull(df), None)

In [285]:
def populate_subvecs_table():
    insert_subvec_query = 'INSERT INTO sub_vecs (tableId, rowIndex, rowCounter, sub_vec) VALUES (%s, %s, %s, %s);'
    data = read_data()
    with engine.connect() as conn:
        for row in data.itertuples():
            if row.subvec is None:
                continue
            for vec in row.subvec:
                conn.execute(insert_subvec_query, (row.tableId, row.rowIndex, row.rowCounter, vec))
    print("Done")

In [281]:
populate_subvecs_table()

OperationalError: (MySQLdb._exceptions.OperationalError) (1048, "Column 'rowCounter' cannot be null")
[SQL: INSERT INTO sub_vecs (tableId, rowIndex, rowCounter, sub_vec) VALUES (%s, %s, %s, %s);]
[parameters: ('0114daa6-c048-4081-b4cf-7a9d7461fa44', 1, None, 'Erosion')]
(Background on this error at: http://sqlalche.me/e/e3q8)